## Install dependencies and setting up GPU Env

In [1]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
OK


In [2]:
!apt-get update & apt-get install tensorflow-model-server

Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:13 https://developer.download.nvid

In [3]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 55kB/s 
     |████████████████████████████████| 368kB 39.4MB/s 
     |████████████████████████████████| 3.2MB 40.9MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0


In [4]:
!pip install requests

## Imports

In [5]:
import os
import json
import random
import requests
import subprocess
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10

%matplotlib inline
tf.__version__

'1.13.1'

## Dataset Preprocessing

load the dataset

In [6]:
(X_train,y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## Defining the model
Using 


In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = "same", activation = "relu", input_shape = [32, 32 , 3]))
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = "same", activation = "relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2, padding = "valid"))
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2, padding = "valid"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.Dense(units=10, activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
__________

## Compiling the model

In [0]:
model.compile(optimizer = 'Adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['sparse_categorical_accuracy'])

### Training model

In [11]:
model.fit(X_train, y_train, batch_size = 128, epochs = 10)

Epoch 1/10
50000/50000 [==============================] - 14s 287us/sample - loss: 14.4970 - sparse_categorical_accuracy: 0.1001
Epoch 2/10
50000/50000 [==============================] - 11s 221us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 3/10
50000/50000 [==============================] - 11s 220us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 4/10
50000/50000 [==============================] - 11s 219us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 5/10
50000/50000 [==============================] - 11s 220us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 6/10
50000/50000 [==============================] - 11s 220us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 7/10
50000/50000 [==============================] - 11s 222us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000
Epoch 8/10
50000/50000 [==============================] - 11s 222us/sample - loss: 14.5063 - spar

### model evaluation

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 139us/sample - loss: 14.5063 - sparse_categorical_accuracy: 0.1000


In [13]:
print(f"test accuracy : {test_accuracy*100}%")

test accuracy : 10.000000149011612%


## Saving Model for Production

###Creating the directory for the model

In [0]:
MODEL_DIR = "model/"
version = 1

In [15]:
export_path = os.path.join(MODEL_DIR, str(version))
export_path

'model/1'

In [0]:
if os.path.isdir(export_path):
  !rm - r (export_path):

### Saving the model for the TensorFlow Serving

In [17]:
tf.saved_model.simple_save(tf.keras.backend.get_session(), export_dir=export_path, inputs={"input_image":model.input}, outputs={t.name:t for t in model.outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: model/1/saved_model.pb


##Setting up the production environment 

### Exporing the MODEL_DIR to enviornment variables. 

In [0]:
os.environ['Model_DIR'] = os.path.abspath(MODEL_DIR)

###Running TensorFlow Serving REST API

In [19]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [20]:
!tail server.log

Failed to start server. Error: Invalid argument: Both server_options.model_base_path and server_options.model_config_file are empty!


## Create the first POST request

In [21]:
random_image = np.random.randint(0,len(X_test))
random_image

6505

### Creating the JSON data object

In [0]:
data = json.dumps({"signature_name":"serving_default", "instances":[X_test[random_image].tolist()]})

In [23]:
data

'{"signature_name": "serving_default", "instances": [[[[161, 134, 80], [156, 127, 75], [142, 116, 70], [138, 118, 73], [150, 135, 81], [135, 126, 69], [142, 128, 73], [150, 128, 75], [156, 126, 77], [162, 131, 80], [173, 146, 85], [163, 140, 79], [145, 126, 71], [129, 114, 67], [124, 110, 67], [136, 113, 73], [144, 118, 79], [127, 106, 67], [111, 96, 61], [100, 87, 56], [98, 85, 55], [114, 101, 70], [106, 94, 63], [109, 97, 62], [122, 110, 64], [143, 130, 74], [161, 146, 87], [149, 132, 77], [131, 118, 70], [120, 110, 67], [112, 104, 69], [97, 92, 65]], [[165, 136, 78], [170, 140, 79], [158, 132, 76], [141, 121, 71], [140, 124, 80], [127, 116, 70], [124, 109, 64], [128, 106, 63], [135, 109, 66], [141, 118, 71], [137, 117, 70], [129, 113, 65], [119, 107, 59], [110, 99, 55], [108, 94, 56], [119, 98, 66], [122, 102, 73], [102, 90, 61], [83, 76, 48], [76, 71, 44], [82, 78, 50], [90, 87, 59], [86, 83, 55], [95, 92, 61], [105, 102, 63], [118, 113, 64], [128, 121, 69], [115, 107, 60], [109, 1

Sending the first POST request to the model

In [0]:
headers = {"content-type":"application/json"}

In [25]:
json_resposne = requests.post(url="http://localhost:8501/v1/models/cifar10:predict", data=data, headers = headers)

ConnectionError: ignored

In [0]:
json_response

In [0]:
predictions = json.loads(json_response.text)['predictions']

In [0]:
predictions

In [0]:
plt.imshow(X_test[random_image])

In [0]:
predictions

In [0]:
ply.imshow(X_test[ransom_image])

In [0]:
class_names[np.argmax(predictions[0])]

## Sending POST request to a specific model

In [0]:
specific_json_response = requests.post(url="http://localhost:8501/v1/model/cifar10/versions/1:predict",data=data,headers=headers)

In [0]:
specific_json_response